In [1]:
import os

In [2]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False
print(IN_COLAB)

True


In [3]:
!pip install graph_nets  dm-sonnet tensorflow_probability

     |████████████████████████████████| 76 kB 2.2 MB/s 
     |████████████████████████████████| 254 kB 9.4 MB/s 
  Created wheel for graph-nets: filename=graph_nets-1.1.0-py3-none-any.whl size=91857 sha256=e6477832b1ef4738b7f6530ab16559ce96c4847b85d6d31bb8a31e874f416970
  Stored in directory: /root/.cache/pip/wheels/44/3d/65/f2e8f0a8d0b28bea5f168fc717261a67303d2183a3e450c812
Successfully built graph-nets


In [4]:
if IN_COLAB and 'drive' not in os.listdir("/content"):
    from google.colab import drive

    drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
if IN_COLAB:
    os.chdir('/content/drive/MyDrive/bakalarka')

In [52]:
import tensorflow as tf

import time
import sys
import re
import pickle

from graph_nets import utils_tf

import dataset

from common import NodeType
from dataset import load_dataset, add_targets, triangles_to_edges, prepare,split_dataset ,add_targets, add_batch

In [7]:
from EncodeProcessDecodeBasic import EncodeProcessDecode
from EncodeProcessDecodeNorm import EncodeProcessDecode as EncodeProcessDecodeNorm
from EncodeProcessDecodeAddNorm import EncodeProcessDecode as EncodeProcessDecodeAddNorm
from EncodeProcessDecodeNoLib import EncodeProcessDecode as EncodeProcessDecodeNoLib
from EncodeProcessDecodeMultinet import EncodeProcessDecode as EncodeProcessDecodeMultinet
from EncodeProcessDecodeNamedMultinet import EncodeProcessDecode as EncodeProcessDecodeNamedMultinet

In [8]:
data_path = 'data/airfoil'
data_train = 'small' if IN_COLAB else 'train'
data_valid = 'small' if IN_COLAB else 'valid'
data_test = 'small' if IN_COLAB else 'test'

In [9]:
mod = "named"
models = {"named": {"model": EncodeProcessDecodeNamedMultinet,
                    "args": {"steps": 15,
                             "learn_features": 2,
                             "n_layers": 2,
                             "lat_size": 128,
                             "edge_feat_cnt": 3,
                             "node_feat_cnt": 5},
                    "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                    size=2, batch=2)],
                    "path": "namednet"},
          "named18": {"model": EncodeProcessDecodeNamedMultinet,
                      "args": {"steps": 18,
                               "learn_features": 2,
                               "n_layers": 2,
                               "lat_size": 128,
                               "edge_feat_cnt": 3,
                               "node_feat_cnt": 5},
                      "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                      size=2, batch=2)],
                      "path": "namednet18"},
          "named12": {"model": EncodeProcessDecodeNamedMultinet,
                      "args": {"steps": 12,
                               "learn_features": 2,
                               "n_layers": 2,
                               "lat_size": 128,
                               "edge_feat_cnt": 3,
                               "node_feat_cnt": 5},
                      "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                      size=2, batch=2)],
                      "path": "namednet12"
                      },
          "named-full": {"model": EncodeProcessDecodeNamedMultinet,
                         "args": {"steps": 12,
                                  "learn_features": 4,
                                  "n_layers": 2,
                                  "lat_size": 128,
                                  "edge_feat_cnt": 3,
                                  "node_feat_cnt": 7},
                         "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                         size=2, batch=2),
                                    dict(noise=0.02, gamma=1.0, field='pressure', history=False,
                                         size=1, batch=2),
                                    dict(noise=0.02, gamma=1.0, field='density', history=False,
                                         size=1, batch=2)],
                         "path": "fullnet"},
          "named12n": {"model": EncodeProcessDecodeNamedMultinet,
                    "args": {"steps": 12,
                             "learn_features": 2,
                             "n_layers": 2,
                             "lat_size": 128,
                             "edge_feat_cnt": 3,
                             "node_feat_cnt": 5},
                    "params": [dict(noise=1.0, gamma=1.0, field='velocity', history=False,
                                    size=2, batch=2)]},
                    "path": "namednet12n"}

In [10]:
params = models[mod]["params"]
targets = tuple([x["field"] for x in params])

In [11]:
dt2 = load_dataset(data_path, data_train)
dt2 = add_targets(dt2, targets, add_history=params[0]['history'])
qq = iter(dt2).next()

In [12]:
NodeTypeCnt = tf.unique(tf.reshape(qq['node_type'][0], qq['node_type'][0].shape[:1])).y.shape[0]

In [13]:
@tf.function
def toGraphsTuple(d, targets=('velocity',)):
    send, recive = triangles_to_edges(d['cells'])
    rel_pos = (tf.gather(d['mesh_pos'], send) - tf.gather(d['mesh_pos'], recive))
    nodes_unique = tf.unique_with_counts(tf.reshape(d["node_type"], [-1]))
    one_hot = tf.one_hot(nodes_unique.idx, NodeTypeCnt, dtype=tf.float32)
    dd = {
        "nodes": tf.concat([*[d[x] for x in targets], one_hot], 1),
        # on change update loss function ^
        "senders": send,
        "receivers": recive,
        "edges": tf.concat([
            rel_pos,
            tf.norm(rel_pos, axis=-1, keepdims=True)], 1)
    }
    return utils_tf.data_dicts_to_graphs_tuple([dd])

In [14]:
def get_init_grp(qq,targets=('velocity',)):
  qqq = {}
  for i, j in qq.items():
      qqq[i] = j[0]
  return toGraphsTuple(qqq, targets)

In [15]:
@tf.function
def rollout(model, grp_,length, loss_mask):
  res = [grp_, ]
  for i in range(length):
      grp2_ = model(grp_, False)
      grp_ = grp_.replace(nodes=tf.where(loss_mask, grp2_.nodes, grp_.nodes))
      res.append(grp_)
  return res

In [16]:
md = models[mod]
model = md["model"](**md["args"])
bsize = 5
mx_iter = 10

In [17]:
dtTrain = load_dataset(data_path,data_train)
dtValid = load_dataset(data_path,data_valid)
dtTrain = add_batch(prepare(split_dataset(add_targets(dtTrain,targets,False)),100 if IN_COLAB else 1000),bsize)

In [18]:
chck_root = "models/" + md["path"] + "/"
chck_name = mod

chck = tf.train.Checkpoint(module=model)

latest = tf.train.latest_checkpoint(chck_root)
if latest is not None:
    print("loading", latest)
    chck.restore(latest)

loading models/namednet/named-197


In [20]:
models_ = [re.match('\w+-\d+',x)[0] for x in filter(lambda x: x !="checkpoint", os.listdir(chck_root))]

In [21]:
itTrain = iter(dtTrain)
itValid = iter(dtValid)

In [22]:
kk = itValid.next()

In [24]:
loss_mask = tf.logical_or(tf.equal(kk['node_type'][0,:,0], NodeType.NORMAL),
                            tf.equal(kk['node_type'][0,:,0], NodeType.OUTFLOW))
loss_mask = tf.reshape(tf.concat([loss_mask for _ in range(md['args']['node_feat_cnt'])], -1), [-1, md['args']['node_feat_cnt']])

In [26]:
chck = tf.train.Checkpoint(module=model)
poss_steps = [x for x in [1, 10, 20, 50, 100, 200] if x <= mx_iter]
stats = {}
losses = {}
for mid in models_:
  chck.restore(chck_root + mid)
  tr = itTrain.next()
  gr = toGraphsTuple(tr,targets)
  loss = model.loss(gr,tr,targets) / bsize
  losses[mid] = loss
  itValid = iter(dtValid)
  eval = {x:0 for x in poss_steps}
  for val in itValid:
    rr = rollout(model,get_init_grp(val,targets), mx_iter,loss_mask)
    pred = tf.concat([x.nodes[...,:model.learn_features] for x in rr], axis=0)
    correct = tf.concat([val[x] for x in targets],axis=2)
    pred = tf.reshape(pred,[mx_iter+1,*correct.shape[1:]])
    error = tf.reduce_mean((pred - correct[:mx_iter+1])**2, axis=-1)
    for x in poss_steps:
      eval[x] += tf.reduce_mean(error[1:x+1])
  stats[mid] = eval
  print(mid)
  

named-11
named-11
named-17
named-17
named-26
named-26
named-27
named-27
named-128
named-128
named-42
named-42
named-81
named-81
named-129
named-129
named-156
named-156
named-197
named-197


In [49]:
ref = [{"name": k,
 "i": int(re.search("\d+",k)[0])*40000,
 "loss": losses[k].numpy(),
 "stat": {i: j.numpy() for i,j in v.items()}} for k,v in stats.items()]

In [53]:
with open("results/" + mod + ".txt",'wb') as x:
  pickle.dump(ref,x)